## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,2021-12-23 23:26:39,-54.8000,-68.3000,56.86,67,75,12.66
1,1,Kapaa,US,2021-12-23 23:29:07,22.0752,-159.3190,78.78,73,75,4.00
2,2,Butaritari,KI,2021-12-23 23:31:37,3.0707,172.7902,81.59,79,47,9.48
3,3,Cayenne,GF,2021-12-23 23:31:37,4.9333,-52.3333,80.10,89,90,4.00
4,4,Rikitea,PF,2021-12-23 23:31:37,-23.1203,-134.9692,77.52,70,34,7.52


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the ideal minimum temperature for your vacation "))
max_temp = float(input("Please enter the ideal maximum temperature for your vacation "))


Please enter the ideal minimum temperature for your vacation 75
Please enter the ideal maximum temperature for your vacation 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kapaa,US,2021-12-23 23:29:07,22.0752,-159.3190,78.78,73,75,4.00
2,2,Butaritari,KI,2021-12-23 23:31:37,3.0707,172.7902,81.59,79,47,9.48
3,3,Cayenne,GF,2021-12-23 23:31:37,4.9333,-52.3333,80.10,89,90,4.00
4,4,Rikitea,PF,2021-12-23 23:31:37,-23.1203,-134.9692,77.52,70,34,7.52
6,6,Idenao,CM,2021-12-23 23:31:38,4.2475,9.0047,78.24,80,100,0.98
7,7,Boyolangu,ID,2021-12-23 23:31:39,-8.1181,111.8935,76.66,90,100,3.18
17,17,Ahuimanu,US,2021-12-23 23:31:43,21.4447,-157.8378,82.11,55,40,7.00
22,22,Zemio,CF,2021-12-23 23:31:45,5.0314,25.1361,78.01,31,94,4.59
25,25,Kismayo,SO,2021-12-23 23:31:46,-0.3582,42.5454,79.93,81,96,10.07
36,36,Hithadhoo,MV,2021-12-23 23:30:37,-0.6000,73.0833,81.19,72,40,2.30


In [6]:
# Count the rows in clean_df
clean_df.count()

City_ID       185
City          185
Country       185
Date          185
Lat           185
Lng           185
Max Temp      185
Humidity      185
Cloudiness    185
Wind Speed    185
dtype: int64

In [7]:
# 4a. Determine if there are any empty rows.
nan_col=[]
for col in clean_df.columns:
    if clean_df.loc[clean_df[col].isnull()].empty !=True:
        nan_col.append(col)

nan_col.count(col)

0

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaner_df = clean_df.dropna()
cleaner_df.count()

City_ID       185
City          185
Country       185
Date          185
Lat           185
Lng           185
Max Temp      185
Humidity      185
Cloudiness    185
Wind Speed    185
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaner_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kapaa,US,78.78,22.0752,-159.3190,
2,Butaritari,KI,81.59,3.0707,172.7902,
3,Cayenne,GF,80.10,4.9333,-52.3333,
4,Rikitea,PF,77.52,-23.1203,-134.9692,
6,Idenao,CM,78.24,4.2475,9.0047,
7,Boyolangu,ID,76.66,-8.1181,111.8935,
17,Ahuimanu,US,82.11,21.4447,-157.8378,
22,Zemio,CF,78.01,5.0314,25.1361,
25,Kismayo,SO,79.93,-0.3582,42.5454,
36,Hithadhoo,MV,81.19,-0.6000,73.0833,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):

        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# Inspect hotel_df
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kapaa,US,78.78,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,Butaritari,KI,81.59,3.0707,172.7902,Isles Sunset Lodge
3,Cayenne,GF,80.10,4.9333,-52.3333,Hôtel Le Dronmi
4,Rikitea,PF,77.52,-23.1203,-134.9692,People ThankYou
6,Idenao,CM,78.24,4.2475,9.0047,Moland Beach Resort
7,Boyolangu,ID,76.66,-8.1181,111.8935,Yos ega cv
17,Ahuimanu,US,82.11,21.4447,-157.8378,Paradise Bay Resort
22,Zemio,CF,78.01,5.0314,25.1361,OCHA Office
25,Kismayo,SO,79.93,-0.3582,42.5454,Agan-Hotel
36,Hithadhoo,MV,81.19,-0.6000,73.0833,Scoop Guest House


In [12]:
# 7. Drop the rows where there is no Hotel Name.
new_hotel_df = hotel_df[hotel_df['Hotel Name']!=""]
new_hotel_df.count()

City          180
Country       180
Max Temp      180
Lat           180
Lng           180
Hotel Name    180
dtype: int64

In [13]:
new_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kapaa,US,78.78,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,Butaritari,KI,81.59,3.0707,172.7902,Isles Sunset Lodge
3,Cayenne,GF,80.10,4.9333,-52.3333,Hôtel Le Dronmi
4,Rikitea,PF,77.52,-23.1203,-134.9692,People ThankYou
6,Idenao,CM,78.24,4.2475,9.0047,Moland Beach Resort
...,...,...,...,...,...,...
646,Parrita,CR,82.17,9.5201,-84.3225,Beso del Viento
650,Swan Hill,AU,83.86,-35.3378,143.5544,Lazy River Motor Inn
653,Beira,MZ,78.57,-19.8436,34.8389,Beira Terrace Hotel
659,Otane,NZ,79.39,-39.8833,176.6333,Ludlow Farmstay


In [14]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
new_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = new_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 

locations = new_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))